In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score
import numpy as np

train=pd.read_csv('data/train.tsv', sep='\t',header=0)
test=pd.read_csv('data/test.tsv', sep='\t',header=0)

In [2]:
X_train = train[['sentence']]
y_train = train.label
X_test = test[['sentence']]
y_test = test.label

In [3]:
import jieba
X_train['cutted_comment'] = X_train.sentence.apply(lambda x: " ".join(jieba.cut(x)))
X_test['cutted_comment'] = X_test.sentence.apply(lambda x: " ".join(jieba.cut(x)))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ALICKZ~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.615 seconds.
Prefix dict has been built succesfully.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
X_test.cutted_comment[:5]

0    派思 股份 与 自贡 华燃 之间 的 交易 ， 便是 根据 辽宁 众华 出具 的 上述 评估...
1    市场 认为 ， 从 财务数据 来看 ， 宣亚 的 收购 是 划算 的 。 宣亚 2016 年...
2    对于 业绩 下滑 ， 智慧 松德 给出 的 解释 是 ， 承担 公司 主要 智能 装备 业务...
3    此前 ， 不少 投资者 在 股 吧 中 讨论 认为 ， 甲基 卡锂 矿复产 将 对 融捷 股...
4    问及 信 达 生物 此前 主动 撤回 信迪 单抗 注射液 上市 申请 是否 影响 其 港股 ...
Name: cutted_comment, dtype: object

In [5]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list
stop_words_file = "stopwordsHIT.txt"
stopwords = get_custom_stopwords(stop_words_file)

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from pprint import pprint
pipe = Pipeline([
    ('vect', CountVectorizer(token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b', 
                       stop_words=frozenset(stopwords))),
    ('tfidf', TfidfTransformer()),
    ('lr', LogisticRegression()),
])

In [7]:
parameters = {
    'vect__max_df': [0.5,0.8, 1.0],
    'vect__max_features': (None, 3, 5),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
}
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipe, parameters, scoring='f1_macro', cv=5, n_jobs=-1, verbose=1)

In [8]:
grid_search.fit(X_train.cutted_comment, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   32.6s finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silenc

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                            

In [9]:
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.660
Best parameters set:
	tfidf__norm: 'l2'
	tfidf__use_idf: True
	vect__max_df: 0.5
	vect__max_features: None
	vect__ngram_range: (1, 1)


In [10]:
y_pre = grid_search.predict(X_test.cutted_comment)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pre))

              precision    recall  f1-score   support

           0       0.68      0.67      0.68       725
           1       0.77      0.46      0.58       368
           2       0.74      0.87      0.80       907

    accuracy                           0.72      2000
   macro avg       0.73      0.67      0.68      2000
weighted avg       0.72      0.72      0.71      2000



In [11]:
y_pre = grid_search.predict(X_train.cutted_comment)
from sklearn.metrics import classification_report
print(classification_report(y_train, y_pre))


              precision    recall  f1-score   support

           0       0.81      0.80      0.81      2157
           1       0.87      0.61      0.71      1072
           2       0.83      0.94      0.88      2771

    accuracy                           0.83      6000
   macro avg       0.84      0.78      0.80      6000
weighted avg       0.83      0.83      0.82      6000

